In [4]:
import argparse
import yaml
from pathlib import Path
from typing import Dict, Any
import re
from tqdm import tqdm 

import pandas as pd
import torch
from transformers import AutoTokenizer

import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data.preprocessor import parse_problems_column, add_choices_len
from src.prompt.prompt_builder import PromptBuilder, PromptConfig
from src.training.model_loader import ModelConfig, load_model_inference

In [5]:
def create_configs(cfg_dict: Dict[str, Any]) -> tuple:
    model_cfg_dict = cfg_dict["model"].copy()
    model_cfg_dict["use_gradient_checkpointing"] = False
    model_cfg = ModelConfig(**model_cfg_dict)
    
    prompt_dict = cfg_dict["inference"]["prompt"]
    prompt_cfg = PromptConfig(
        policy=prompt_dict["policy"],
        mode="test",
        verbose=False
    )
    
    inference_cfg = cfg_dict.get("inference", {})
    
    return model_cfg, prompt_cfg, inference_cfg

In [7]:
with open("../../config_ax.yaml", "r") as f:
    cfg_dict = yaml.safe_load(f)

model_cfg, prompt_cfg, inference_cfg = create_configs(cfg_dict)

adapter_path = inference_cfg["adapter_path"]
output_path = inference_cfg["output_path"]
output_logits_path = inference_cfg["output_logits_path"]
test_data_path = inference_cfg["test_data_path"]
max_new_tokens = inference_cfg.get("max_new_tokens", 100)

In [8]:
def load_test_data(test_path: Path) -> pd.DataFrame:
    """Load and preprocess test data."""
    test_df = pd.read_csv(test_path)
    test_df = parse_problems_column(test_df)
    test_df = add_choices_len(test_df)
    return test_df

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}\n")

print(f"Loading test data from {test_data_path}...")
test_df = load_test_data("../../" + test_data_path)
print(f"Loaded {len(test_df)} rows\n")

Device: cuda

Loading test data from ./data/test.csv...
Loaded 869 rows



In [11]:
test_df = test_df.iloc[:5]

In [12]:
print(f"Loading tokenizer from {model_cfg.model_name_or_path}...")
tokenizer = AutoTokenizer.from_pretrained(
    model_cfg.model_name_or_path,
    trust_remote_code=model_cfg.trust_remote_code,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading tokenizer from skt/A.X-4.0-Light...


In [14]:
print(f"Loading model from {adapter_path}...")
model = load_model_inference(model_cfg, "../../" + adapter_path)
model.eval()
print("Model loaded successfully!\n")

Loading model from ./models/A.X-4.0-Light/final_model...
Loading Base Model for Inference: skt/A.X-4.0-Light


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading LoRA Adapter from: ../.././models/A.X-4.0-Light/final_model
Model loaded successfully!



In [15]:
builder = PromptBuilder(prompt_cfg)
print("PromptBuilder ready!\n")

PromptBuilder ready!



In [16]:
row_dict = test_df.iloc[0].to_dict()

k = int(row_dict["choices_len"])

output = builder.build_message(row_dict)
messages = output["messages"]

prompt_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    return_dict_in_generate=True,
    output_scores=True,
)

inputs = tokenizer(
    prompt_text,
    return_tensors="pt",
    truncation=True,
    max_length=4096
).to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_scores=True,
    )

In [18]:
output_ids.sequences

tensor([[    25,      9,  27010,  22290,   1122,   7517,  81183,   3800,   1251,
          49185,   8379,    506,    879,     54,    239,    308,  10803,  14181,
           7517,  11804,    920,  40652,  28274,   2518,    506,  11806,  64660,
           1694,     54,    239,  24223,   9200,   8254,  70532,  93225,     52,
          29650,    579,   3597, 101115,  25067,  68241,   5226,  29020,     54,
             27,     25,      8,  12961,  40652,    239,  16662,  12882,   6468,
          15230,   2399,    746,  12615,  41230,    804,   1253,    298,   2319,
          11535,  41230,   1025,   2178,  29988,  18272,    462,   4118,   1025,
           6051,  19609,    349,    261,  41230,    878,    308,    791,     54,
           2319,   2276,  68896,   1877,  50623,  60960,  28347,   3606,   1211,
           1253,    261,   2319,   4392,    298,   1336,   6222,  64016,  87449,
          10585,  14104,    804,   1253,    261,   2319,   2276,   5981,  27906,
            298,   2681,    

In [19]:
generated_ids = output_ids.sequences[0][inputs['input_ids'].shape[1]:]
print(tokenizer.convert_ids_to_tokens(generated_ids))

['5', '<|im_end|>']


In [22]:
full_text = tokenizer.decode(output_ids.sequences[0], skip_special_tokens=False)
print(f"전체 텍스트:\n{full_text}")

전체 텍스트:
<|im_start|><|system|>당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.
이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.
당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|><|im_start|><|user|>### 지문
사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다.   독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의  즐거움’이 있다. 독자는 독서를 통해 책과 소통하는 즐거움을 경험한다.  독서는   필자와 간접적으로 대화하는 소통 행위이다.  독자는 자신이 속한   사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는  사회나 시대를 경험한다.  직접 경험하지 못했던 다양한 삶을  필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를   바라볼 수 있다.  이때 같은 책을 읽은 독자라도 독자의 배경 지식이나 관점 등의 독자 요인,  읽기 환경이나 과제 등의 상황  요인이 다르므로,  필자가 보여 주는 세계를 그대로 수용하지  않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다. 이러한 소통은 독자가 책의 내용에 대해 질문하고 답을  찾아내는 과정에서 가능해진다.  독자는 책에서 답을 찾는  질문,  독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다.   전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고,   책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로  구성할 수도 있다.  또한 후자의 경우 책에는 없는 독자의  경험에서 답을 찾을 수 있다.  이런 질문들을 풍부히 생성 하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다. 한편 독자는 ㉠다른 독자와 소통하는 즐거움 을 경험할 수도  있다.  책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임 이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당 한다.  비슷한 해석에 서로 공감하며 기존 인식을 강화하거나  관점의 차